In [1]:
import json
import numpy as np
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import pickle
import os
import math
import random
from eoh.problems.optimization.classic_benchmark_path_planning.utils.benchmark import MultiMapBenchmarker
import warnings
import types
import sys

In [2]:
class Map(NamedTuple):
    grid: np.ndarray
    start: Union[Tuple[float, float], Tuple[float, float, float]]
    goal: Union[Tuple[float, float], Tuple[float, float, float]]
    obstacles: List[Union[Tuple[float, float, float, float], Tuple[float, float, float, float, float, float]]] # x, y, width, height or x, y, z, width, height, dimension
    size: Union[Tuple[int, int], Tuple[int, int, int]]

class PlannerResult(NamedTuple):
    success: bool
    path: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    nodes: List[Union[Tuple[float, float], Tuple[float, float, float]]]
    edges: List[Tuple[Tuple[float, ...], Tuple[float, ...]]]  # (parent, child)

class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position
        self.parent = parent
        self.cost = cost
        self.children = []
        self.valid = True  # 장애물 충돌 여부 등


In [3]:
def visualize_map_shapes(
    map_array: np.ndarray,
    start: Optional[Tuple[int, ...]] = None,
    goal: Optional[Tuple[int, ...]] = None,
    obs:List[Tuple[int, ...]] = None,
    path: Optional[List[Tuple[float, ...]]] = None,
    nodes: Optional[List[Tuple[float, ...]]] = None,
    edges: Optional[List[Tuple[float, ...]]] = None,
    title: str = "Map Visualization"
):
    fig = go.Figure()

    if map_array.ndim == 2:
        height, width = map_array.shape
        
        for x, y, w, h in obs:
                fig.add_shape(
                    type="rect",
                    x0=x, x1=x+w, y0=y, y1=y+h,
                    fillcolor="purple",opacity=0.5,
                    line=dict(width=0)
                )

        # 방문 노드
        if nodes:
            vx, vy = zip(*nodes)
            fig.add_trace(go.Scatter(
                x=vx, y=vy, mode="markers",
                marker=dict(size=4, color="blue"),
                name="nodes"
            ))

        # 경로
        if path:
            px, py = zip(*path)
            fig.add_trace(go.Scatter(
                x=px, y=py, mode="lines+markers",
                line=dict(color="green"),
                marker=dict(size=6),
                name="Path"
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter(
                    x=[parent.position[0], child.position[0]], y=[parent.position[1], child.position[1]],
                    mode="lines",
                    line=dict(color="lightblue", width=1),
                    showlegend=False,
                    hoverinfo="skip"
                ))


        # 시작/목표
        if start:
            fig.add_trace(go.Scatter(
                x=[start[0]], y=[start[1]], mode="markers",
                marker=dict(size=10, color="red"),
                name="Start"
            ))

        if goal:
            fig.add_trace(go.Scatter(
                x=[goal[0]], y=[goal[1]], mode="markers",
                marker=dict(size=10, color="orange"),
                name="Goal"
            ))

        fig.add_shape(
            type="rect",
            x0=0, y0=0,
            x1=width, y1=height,
            line=dict(color="white", width=3),
            fillcolor="rgba(0,0,0,0)",  # 투명 내부
            layer="above"
        )

        fig.update_layout(
            title=title,
            xaxis=dict(scaleanchor="y", showgrid=False),
            # yaxis=dict(showgrid=False, autorange="reversed"),
            yaxis=dict(showgrid=False),
            height=600, width=600
        )

    elif map_array.ndim == 3:
        z, y, x = map_array.nonzero()
        x, y, z = list(x), list(y), list(z)

        # for x,y,w,h in obs:
        #     fig.add_trace(go.Mesh3d(
        #         x=x, y=y, z=z,
        #         color='black',
        #         opacity=1.0,
        #         alphahull=0,
        #         name='Obstacles'
        #     ))

        fig.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            color='black',
            opacity=1.0,
            alphahull=0,
            name='Obstacles'
        ))

        if nodes:
            vx, vy, vz = zip(*nodes)
            fig.add_trace(go.Scatter3d(
                x=vx, y=vy, z=vz,
                mode='markers',
                marker=dict(size=2, color='blue'),
                name='Visited'
            ))

        # 엣지 (연결 정보)
        if edges:
            for parent, child in edges:
                fig.add_trace(go.Scatter3d(
                    x=[parent.position[0], child.position[0]],
                    y=[parent[1].position, child.position[1]],
                    z=[parent[2].position, child.position[2]],
                    mode='lines',
                    line=dict(color='lightblue', width=2),
                    showlegend=False,
                    hoverinfo="skip"
                ))

        if path:
            px_, py_, pz_ = zip(*path)
            fig.add_trace(go.Scatter3d(
                x=px_, y=py_, z=pz_,
                mode='lines+markers',
                marker=dict(size=3, color='green'),
                name='Path'
            ))

        if start:
            fig.add_trace(go.Scatter3d(
                x=[start[0]], y=[start[1]], z=[start[2]],
                mode='markers',
                marker=dict(size=5, color='red'),
                name='Start'
            ))

        if goal:
            fig.add_trace(go.Scatter3d(
                x=[goal[0]], y=[goal[1]], z=[goal[2]],
                mode='markers',
                marker=dict(size=5, color='orange'),
                name='Goal'
            ))

        fig.update_layout(
            title=title,
            scene=dict(aspectmode='data'),
            height=700, width=700
        )

        

    fig.show()


In [4]:
class MapIO:
    @staticmethod
    def save_map(map_data: Map, filename: str) -> None:
        """Save Map object to a binary file."""
        with open(filename, 'wb') as f:
            pickle.dump(map_data, f)

    @staticmethod
    def load_map(filename: str) -> Map:
        """Load Map object from a binary file."""
        if not os.path.exists(filename):
            raise FileNotFoundError(f"Map file not found: {filename}")
        with open(filename, 'rb') as f:
            return pickle.load(f)

In [5]:
raw_maps = ["Maze_map_easy.pkl", "Narrow_map.pkl", "Multi_obs_map.pkl"]

multi_obs_map = MapIO.load_map("Multi_obs_map.pkl")
print("Start:", multi_obs_map.start)
print("Obstacles:", len(multi_obs_map.obstacles))
print(multi_obs_map.grid.shape)

hard_maze_map = MapIO.load_map("Maze_map.pkl")
print("Start:", hard_maze_map.start)
print("Obstacles:", len(hard_maze_map.obstacles))
print(hard_maze_map.grid.shape)


maze_map = MapIO.load_map("Maze_map_easy.pkl")
print("Start:", maze_map.start)
print("Obstacles:", len(maze_map.obstacles))
print(maze_map.grid.shape)

narrow_map = MapIO.load_map("Narrow_map.pkl")
print("Start:", narrow_map.start)
print("Obstacles:", len(narrow_map.obstacles))
print(narrow_map.grid.shape)

Start: (1, 1)
Obstacles: 35
(100, 100)
Start: (29, 40)
Obstacles: 8
(100, 100)
Start: (29, 10)
Obstacles: 5
(100, 100)
Start: (80, 50)
Obstacles: 5
(100, 100)


In [6]:
maps=[multi_obs_map, hard_maze_map, maze_map, narrow_map]

In [7]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method__.json'
with open(json_path, "r") as f:
    classic_method = json.load(f)

print(classic_method[0].keys())

dict_keys(['algorithm', 'algorithm_description', 'planning_mechanism', 'code'])


In [14]:
print([n['algorithm'] for n in classic_method])

['RRT', 'RRT*', 'RRT-Connect', 'RRT*-Connect', 'BI-RRT', 'BI-RRT*', 'Informed-RRT*', 'Informed-RRT*-Connect', 'Bidirectional-Informed-RRT*', 'Improved-RRT*-Connect']


In [21]:
print(classic_method[-1]['code'])


class Node:
    def __init__(self, position, parent=None, cost=0.0):
        self.position = position        # Tuple[float, ...] → 2D: (x,y), 3D: (x,y,z)
        self.parent = parent            # Node or None
        self.cost = cost                # Path cost from its tree root
        self.children = []
        self.valid = True               # For collision checking etc.

    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self

    def path_from_root(self):
        path = []
        n = self
        while n:
            path.append(n.position)
            n = n.parent
        return path[::-1]


class Planner:
    def __init__(
        self,
        max_iter: int = 5000,
        step_size: float = 5.0,
        post_opt_iters: int = 400,   # extra iterations after first solution
        max_no_improve: int = 150,   # early stop if no improvement for this many iterations
        improve_tol: float = 1e-6,   # minimal cost drop to

In [11]:
import_string ='''
from typing import Tuple, Literal, Union, Optional, List, Dict, NamedTuple, Callable, Any, Set, TYPE_CHECKING, Type
import time
from queue import Queue
import numpy as np
import random
import math
import sys
import os
from eoh.problems.optimization.classic_benchmark_path_planning.utils.architecture_utils import PlannerResult, Map
'''

In [29]:
code = classic_method[-1]['code']

In [27]:
code_string = code
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    planning_module = types.ModuleType("planning_module")
    exec(import_string+code_string, planning_module.__dict__)
    sys.modules[planning_module.__name__] = planning_module
    p = planning_module.Planner(max_iter=5000)

In [28]:
for pmap in maps:
    result = p.plan(map=pmap)
    visualize_map_shapes(pmap.grid, obs=pmap.obstacles, start=pmap.start, goal=pmap.goal,
                        path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)

In [65]:
json_path = './eoh/src/eoh/problems/optimization/classic_benchmark_path_planning/utils/classic_method_.json'

In [34]:
result = classic_method[7]

In [105]:
result = {'algorithm': 'Improved-RRT*-Connect',
 'algorithm_description': 'This algorithm with post-optimization: a bidirectional RRT*-Connect that, after finding a first feasible “best shot” path, keeps optimizing for only a user-defined number of extra iterations (or until no further improvements occur), then terminates—while maintaining strict collision checks and a 30 s time cap.',
 'planning_mechanism': 'Mechanism: Alternate tree growth with RRT* best-parent selection and local rewiring; greedily connect the opposite tree; upon the first connection, record the best path and continue only N additional iterations (or until M consecutive non-improving attempts), always staying within bounds and validating node and edge collisions; stop early on time limit and return the best path so far.',
 'code': code}

In [35]:
with open(json_path, "a") as f:
    json.dump(result, f, indent=4)
    f.write(",\n")

In [33]:
classic_method[7]['code'] = code

In [58]:
code=''''''

In [18]:
maps = [multi_obs_map, maze_map, narrow_map]
benchmarker = MultiMapBenchmarker(maps=maps, iter=100)

total_df = pd.DataFrame()

# for method in classic_method[:-2]:
for method in classic_method:
    code_string = method['code']
    namedf = pd.DataFrame()
    namedf['alg_name'] = [method['algorithm']]* len(maps)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        planning_module = types.ModuleType("planning_module")
        exec(import_string+code_string, planning_module.__dict__)
        sys.modules[planning_module.__name__] = planning_module
        planner = planning_module.Planner(max_iter=5000)
        res, avg_rest = benchmarker.run(planner.plan)
        if method['algorithm'] == 'RRT':
            ref_avg = avg_rest
            outputs = []
        outputs = benchmarker.get_results()
        if avg_rest is None: continue
        imp_res = MultiMapBenchmarker.get_improvement(ref_avg, avg_rest)

        res_df = pd.concat([namedf, avg_rest, imp_res], axis=1)
        total_df = pd.concat([total_df, res_df], axis=0)

total_df


[2025.09.09 - 15:13:33] Map 1
Iteration 1: Time taken: 0.0275 seconds, Success: True
Iteration 2: Time taken: 0.0190 seconds, Success: True
Iteration 3: Time taken: 0.0191 seconds, Success: True
Iteration 4: Time taken: 0.0200 seconds, Success: True
Iteration 5: Time taken: 0.0126 seconds, Success: True
Iteration 6: Time taken: 0.0110 seconds, Success: True
Iteration 7: Time taken: 0.0121 seconds, Success: True
Iteration 8: Time taken: 0.0504 seconds, Success: True
Iteration 9: Time taken: 0.0266 seconds, Success: True
Iteration 10: Time taken: 0.0571 seconds, Success: True
Iteration 11: Time taken: 0.0250 seconds, Success: True
Iteration 12: Time taken: 0.0245 seconds, Success: True
Iteration 13: Time taken: 0.0240 seconds, Success: True
Iteration 14: Time taken: 0.0487 seconds, Success: True
Iteration 15: Time taken: 0.0301 seconds, Success: True
Iteration 16: Time taken: 0.0156 seconds, Success: True
Iteration 17: Time taken: 0.0182 seconds, Success: True
Iteration 18: Time taken: 0

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
0,RRT,0,1.0,0.042633,461.98,188.080816,0.006254,0.0,-0.000000,-0.000000,0.000000,0.000000
1,RRT,1,1.0,0.345472,1637.59,303.968509,0.003852,0.0,-0.000000,-0.000000,0.000000,0.000000
2,RRT,2,1.0,0.154483,859.41,150.713378,0.007887,0.0,-0.000000,-0.000000,0.000000,0.000000
0,RRT*,0,1.0,0.156539,447.98,157.771191,0.014527,0.0,-267.177534,16.115213,132.299872,-76.268718
1,RRT*,1,1.0,0.972932,1630.00,225.665909,0.011059,0.0,-181.623651,25.760103,187.116241,-48.399494
2,RRT*,2,1.0,0.286043,796.67,116.530178,0.021042,0.0,-85.161796,22.680933,166.805158,-20.178327
0,RRT-Connect,0,1.0,0.017664,106.12,195.985942,0.011055,0.0,58.567055,-4.203047,76.774527,17.113380
1,RRT-Connect,1,1.0,0.033089,236.56,302.131675,0.007242,0.0,90.422151,0.604284,88.002431,27.687514
2,RRT-Connect,2,1.0,0.041473,251.01,160.833474,0.014351,0.0,73.153721,-6.714797,81.963031,21.012972
0,RRT*-Connect,0,1.0,0.024156,129.62,171.903015,0.016822,0.0,43.339729,8.601516,168.993066,15.567187


In [17]:
total_df1 = total_df

In [19]:
total_df1

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
0,RRT,0,1.00,0.035257,461.98,188.080816,0.006254,0.0,-0.000000,-0.000000,0.000000,0.000000
1,RRT,1,0.00,0.383815,1704.46,NaN,1.000000,0.0,-0.000000,NaN,0.000000,NaN
2,RRT,2,1.00,0.214382,1646.97,305.294357,0.003862,0.0,-0.000000,-0.000000,0.000000,0.000000
3,RRT,3,1.00,0.068837,839.78,152.790835,0.007785,0.0,-0.000000,-0.000000,0.000000,0.000000
0,RRT*,0,1.00,0.069093,432.49,157.533630,0.014613,0.0,-95.968119,16.241521,133.670537,-24.873779
1,RRT*,1,0.00,0.648108,1683.19,NaN,1.000000,0.0,-68.859471,NaN,0.000000,NaN
2,RRT*,2,1.00,0.425404,1630.38,225.392740,0.011241,0.0,-98.432276,26.171993,191.051088,-23.340029
3,RRT*,3,1.00,0.125224,800.93,116.560962,0.021343,0.0,-81.914782,23.712072,174.162207,-18.961209
0,RRT-Connect,0,1.00,0.017552,116.02,196.961209,0.011149,0.0,50.216258,-4.721584,78.282236,14.511972
1,RRT-Connect,1,1.00,0.200972,1050.80,532.772541,0.003773,100.0,47.638209,NaN,-99.622699,NaN


In [ ]:
total_df1[total_df['map_id'] == 1]

,alg_name,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
1,RRT,1,0.00,0.383815,1704.46,NaN,1.000000,0.0,-0.000000,NaN,0.000000,NaN
1,RRT*,1,0.00,0.648108,1683.19,NaN,1.000000,0.0,-68.859471,NaN,0.000000,NaN
1,RRT-Connect,1,1.00,0.200972,1050.80,532.772541,0.003773,100.0,47.638209,NaN,-99.622699,NaN
1,RRT*-Connect,1,1.00,0.382580,1019.90,402.654093,0.006680,100.0,0.321630,NaN,-99.331971,NaN
1,BI-RRT,1,1.00,0.122854,997.55,538.847689,0.003646,100.0,67.991237,NaN,-99.635383,NaN
1,BI-RRT*,1,1.00,0.939351,991.02,406.871172,0.006614,100.0,-144.740653,NaN,-99.338567,NaN
1,Informed-RRT*,1,0.42,3.149493,2376.59,373.522374,0.584920,42.0,-720.576273,NaN,-41.508049,NaN
1,Informed-RRT*-Connect,1,0.98,3.011959,1045.81,385.133193,0.035209,98.0,-684.742899,NaN,-96.479126,NaN
1,Bidirectional-Informed-RRT*,1,1.00,0.587318,1044.63,403.448162,0.007151,100.0,-53.021275,NaN,-99.284899,NaN
1,Improved-RRT*-Connect,1,0.92,0.628170,1069.91,396.054457,0.086717,92.0,-63.664824,NaN,-91.328337,NaN


In [20]:
# classic method result
grouped_avg = total_df.groupby('alg_name').mean()
a = pd.DataFrame(grouped_avg)
a

,map_id,success_rate,time_avg,num_nodes_avg,path_length_avg,smoothness_avg,success_improvement,time_improvement,length_improvement,smoothness_improvement,objective_score
alg_name,,,,,,,,,,,
BI-RRT,1.0,1.0,0.016286,155.170000,213.941368,0.010032,0.0,88.471336,-0.007719,67.326861,26.876491
BI-RRT*,1.0,1.0,0.055723,175.796667,186.840439,0.016921,0.0,49.345831,12.775225,180.716786,18.262378
Bidirectional-Informed-RRT*,1.0,1.0,0.149482,500.273333,171.859631,0.029741,0.0,-95.435586,19.815096,389.512360,-22.720095
Improved-RRT*-Connect,1.0,1.0,0.146029,518.830000,170.811093,0.030153,0.0,-62.089841,19.889434,394.092078,-12.678605
Informed-RRT*,1.0,1.0,0.544449,930.400000,161.982690,0.030676,0.0,-513.705904,23.838074,415.824461,-147.265034
Informed-RRT*-Connect,1.0,1.0,0.631165,541.140000,165.450936,0.044705,0.0,-712.473861,22.505155,634.562555,-206.068314
RRT,1.0,1.0,0.180863,986.326667,214.254234,0.005997,0.0,0.000000,0.000000,0.000000,0.000000
RRT*,1.0,1.0,0.471838,958.216667,166.655759,0.015543,0.0,-177.987660,21.518749,162.073757,-48.282179
RRT*-Connect,1.0,1.0,0.050163,227.040000,187.996802,0.016492,0.0,59.113488,11.617421,176.692995,20.940995


In [20]:
(986.32 - 518)/986.32

0.4748154757076811

In [ ]:
results = [outputs[35],outputs[195],outputs[265]]

In [ ]:
for pmap, result in zip(maps, results):
    
    visualize_map_shapes(pmap.grid, obs=pmap.obstacles, start=pmap.start, goal=pmap.goal,
                        path=result.path, nodes=list(map(lambda x: x.position, result.nodes)), edges=result.edges)